In [1]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
 
# Dataset id from huggingface.co/dataset
dataset_id = "DataForGood/climateguard"
 
# Load raw dataset
train_dataset = load_dataset(dataset_id, split='train')

split_dataset = train_dataset.train_test_split(test_size=0.1)
split_dataset['train'][0]


{'label_studio_id': 1739,
 'id': '0dbeca955359710bd676973cc0bfdb5ec2151ff6d7cc0f6a08a88187840cebb7',
 'day': 13,
 'year': 2024,
 'month': 11,
 'start': '2024-11-13T08:52:00',
 'channel': 'rmc',
 'plaintext': "et on est obligé de baisser nos émissions de carbone à moins de vouloir euh avoir des inondations à valence tous les week <unk> end inondations annuelles et ben oui précisément le rapport entre les émissions carbone et les catastrophes naturelles il est il est tout droit il est direct et vu que les jets privés font beaucoup d' émissions de carbone et c' est un lien si elle vient de dire c' est physiquement rien scientifique ça n' a rien à voir si le réchauffement des eaux c' est pas lié est aux pas lié avions d' aux affaires aux freiner avions le d' réchauffement affaires des freiner eaux le ces réchauffement livres des eaux c' est lié au réchauffement climatique il est pinos il est lié aux émissions carbones oui mais tout est peanuts donc si on considère que toute épinette on fai

In [2]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=512, 
    chunk_overlap=128,
)
# chunks = splitter.split_text(record["abstract"])

In [4]:
import random
import pandas as pd

records = []

for record in train_dataset:
    if record["misinformation_claims"]:
        for claim in record["misinformation_claims"]:
            for label in claim["labels"]:
                records.append({
                    "text": claim["text"],
                    "label": 1
                })
    else:
        chunks = splitter.split_text(record["plaintext"])
        records.append({
            "text": random.choice(chunks),
            "label": int(record["misinformation"]),
        })

claims_dataset = Dataset.from_pandas(pd.DataFrame.from_records(records))

In [5]:
split_dataset = claims_dataset.train_test_split(test_size=0.1)
split_dataset['train'][0]

{'text': "avion en fait ils pointent à chaque fois des responsabilités en fait c' est peut être tout simplement l' arrêt la responsabilité d' une personne alors que nous nous passons notre temps à noyer les responsabilités individuelles c' est à dire que si ça se passe en france on a expliqué que c' est parce que le contrôleur travaille trop et que les conditions de travail ne sont pas ben khemis seck et que tout d' un coup le syndicat va monter au créneau pour dire ben oui réagirait chez nous le président ne réagit pas à jouer il y aurait une enquête yves <unk> les conclusions vous n' en nommer que je veux vous dire si je devais gyrophare dans l' état d' esprit de la france qui est le nôtre c' est que tout est inversé ça servirait la cause de ceux qui excite ben oui écoutez la la charge de travail vous rendez pas compte qu' il y a sur ces gens là euh aux médias les treize juges dans tous les domaines de la france pointer répugne les pointer responsabilités individuelles les derrière r

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
 
# Model id to load the tokenizer
model_id = "almanach/camembertav2-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')

def format_labels(example):
    return {"labels": str(int(example["misinformation"]))}
 
# Tokenize dataset
# if "misinformation" in split_dataset["train"].features.keys():
#     split_dataset =  split_dataset.rename_column("misinformation", "labels") # to match Trainer
tokenized_dataset = split_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dataset = tokenized_dataset.map(format_labels, batched=False)
 
tokenized_dataset["train"].features.keys()
# dict_keys(['labels', 'input_ids', 'attention_mask'])


Map:   0%|          | 0/643 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/643 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

dict_keys(['label_studio_id', 'id', 'day', 'year', 'month', 'start', 'channel', 'model_name', 'channel_name', 'model_reason', 'model_result', 'channel_title', 'url_mediatree', 'channel_program', 'plaintext_whisper', 'channel_program_type', 'week_number', 'misinformation', 'cards_claims', 'misinformation_claims', 'comments', 'input_ids', 'attention_mask', 'labels'])

In [12]:
tokenized_dataset["train"].features

{'label_studio_id': Value(dtype='int64', id=None),
 'id': Value(dtype='string', id=None),
 'day': Value(dtype='int64', id=None),
 'year': Value(dtype='int64', id=None),
 'month': Value(dtype='int64', id=None),
 'start': Value(dtype='string', id=None),
 'channel': Value(dtype='string', id=None),
 'model_name': Value(dtype='string', id=None),
 'channel_name': Value(dtype='string', id=None),
 'model_reason': Value(dtype='string', id=None),
 'model_result': Value(dtype='int64', id=None),
 'channel_title': Value(dtype='string', id=None),
 'url_mediatree': Value(dtype='string', id=None),
 'channel_program': Value(dtype='string', id=None),
 'plaintext_whisper': Value(dtype='string', id=None),
 'channel_program_type': Value(dtype='string', id=None),
 'week_number': Value(dtype='int64', id=None),
 'misinformation': Value(dtype='bool', id=None),
 'cards_claims': [{'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
   'text': Value(dtype='string', id=None)}],
 'misinf

In [13]:
# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"
 
# Prepare model labels - useful for inference
labels = list(set(tokenized_dataset["train"]["labels"]))
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
 
# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
tokenized_dataset["train"].to_pandas()[["input_ids"]].map(len).describe()

,input_ids
count,643.0
mean,658.0
std,0.0
min,658.0
25%,658.0
50%,658.0
75%,658.0
max,658.0


In [6]:
import numpy as np
from sklearn.metrics import f1_score
 
# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(
            labels, predictions, labels=labels, pos_label=1, average="weighted"
        )
    return {"f1": float(score) if score == 1 else score}


In [12]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments
 
# Define training args
training_args = TrainingArguments(
    output_dir= "ModernBERT-base-climateguard",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
        num_train_epochs=5,
    bf16=False, # bfloat16 training 
    fp16=False,
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    use_mps_device=True,
    metric_for_best_model="f1",
    # push to hub parameters
    push_to_hub=True,
    hub_strategy="every_save",
    hub_token=HfFolder.get_token(),
)
 
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()
# {'train_runtime': 3642.7783, 'train_samples_per_second': 1.235, 'train_steps_per_second': 0.04, 'train_loss': 0.535627057634551, 'epoch': 5.0}


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [10]:
import platform
platform.platform()


'macOS-14.3-arm64-arm-64bit'

In [11]:
import torch

torch.has_mps

/var/folders/_y/jjyv0bm5597_x15dvpxnsmrc0000gn/T/ipykernel_49241/908756464.py:3: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps


True